In [2]:
!export PYTHONPATH=.:$PYTHONPATH

In [3]:
from odi.model_util import odi_util as outil
from odi.feature_engg import util as cricutil

In [4]:
match_list_df=cricutil.read_csv_with_date('data/csv_load/match_list.csv',date_fied="date")

In [5]:
match_list_df['date'].min()

Timestamp('2011-01-12 00:00:00')

In [11]:
import pandas as pd
match=pd.read_csv('data/csv_load/1000887.csv')
match.head()

,match_id,innings,team,opponent,ball,batsman,non_striker,bowler,scored_runs,extras,total,extra_type,wicket,wicket_type,player_out,fielders,winner
0,1000887,1st innings,Australia,Pakistan,0.1,DA Warner,TM Head,Mohammad Amir,0,0,0,NaN,0,NaN,NaN,NaN,1
1,1000887,1st innings,Australia,Pakistan,0.2,DA Warner,TM Head,Mohammad Amir,0,0,0,NaN,0,NaN,NaN,NaN,1
2,1000887,1st innings,Australia,Pakistan,0.3,DA Warner,TM Head,Mohammad Amir,0,0,0,NaN,0,NaN,NaN,NaN,1
3,1000887,1st innings,Australia,Pakistan,0.4,DA Warner,TM Head,Mohammad Amir,0,0,0,NaN,0,NaN,NaN,NaN,1
4,1000887,1st innings,Australia,Pakistan,0.5,DA Warner,TM Head,Mohammad Amir,0,1,1,wides,0,NaN,NaN,NaN,1


In [13]:
match_stats_df=pd.read_csv('data/csv_load/match_stats.csv')
match_stats_df.head()

,match_id,team_statistics,batsman_1,batsman_1_runs,batsman_2,batsman_2_runs,batsman_3,batsman_3_runs,batsman_4,batsman_4_runs,...,bowler_8,bowler_8_wickets,bowler_9,bowler_9_wickets,bowler_10,bowler_10_wickets,bowler_11,bowler_11_wickets,total_run,total_wickets
0,661693,West Indies,KOA Powell,73,J Charles,31,KA Edwards,123,LMP Simmons,9,...,not_bowled,0,not_bowled,0,not_bowled,0,not_bowled,0,363,10
1,661693,New Zealand,MJ Guptill,6,JD Ryder,17,KS Williamson,16,LRPL Taylor,9,...,not_bowled,0,not_bowled,0,not_bowled,0,not_bowled,0,160,4
2,636159,England,AN Cook,4,JE Root,3,IR Bell,41,GS Ballance,79,...,not_bowled,0,not_bowled,0,not_bowled,0,not_bowled,0,269,4
3,636159,Australia,AJ Finch,121,DA Warner,65,SR Watson,0,MJ Clarke,43,...,not_bowled,0,not_bowled,0,not_bowled,0,not_bowled,0,270,7
4,636160,England,AN Cook,22,IR Bell,68,JE Root,2,GS Ballance,9,...,not_bowled,0,not_bowled,0,not_bowled,0,not_bowled,0,300,9


In [10]:
import pandas as pd
pd.read_csv('data/preprocess/country_rank_2015-12-31.csv').head()

,country,win_ratio,effective_win_by_runs,effective_win_by_wickets,matches_played,win_count,score,rank
0,New Zealand,0.678571,300,38,28,19,3.268329,1
1,Australia,0.823529,687,34,17,14,3.149603,2
2,South Africa,0.700000,1077,10,20,14,2.979461,3
3,Bangladesh,0.687500,264,17,16,11,2.495623,4
4,India,0.619048,187,11,21,13,2.480792,5


In [5]:
import pickle

In [6]:
x=pickle.load(open('retrain_xy/first_innings_base_train_x.pkl','rb'))
y = pickle.load(open('retrain_xy/adversarial_first_innings_train_y.pkl','rb'))


In [15]:
x.shape

(405, 20)

In [69]:
y.shape

(5923,)

In [1]:
import pandas as pd
from odi.feature_engg import util as cricutil
from odi.data_loader import  data_loader as dl
from odi.model_util import odi_util as outil
from odi.feature_engg import feature_extractor as fe

import os
import numpy as np
import pickle
from tqdm import tqdm
import click
import tensorflow as tf

In [ ]:
outil.use_model_from('dev')
train_start_dt = cricutil.str_to_date_time(train_start)
test_start_dt = cricutil.str_to_date_time(test_start)
if test_end is None:
    test_end_dt = cricutil.today_as_date_time()
else:
    test_end_dt = cricutil.str_to_date_time(test_end)

overall_start = train_start_dt
overall_end = test_end_dt
match_list_df = cricutil.read_csv_with_date(dl.CSV_LOAD_LOCATION + os.sep + 'match_list.csv')
match_list_df = match_list_df[(match_list_df['date'] >= overall_start) & \
                              (match_list_df['date'] <= overall_end)]
match_stats_df = pd.read_csv(dl.CSV_LOAD_LOCATION + os.sep + 'match_stats.csv')
match_list_df = match_list_df.merge(match_stats_df, how='inner', on='match_id')
match_id_list = list(match_list_df['match_id'].unique())

feature_list_train = []
target_list_train = []

feature_list_test =[]
target_list_test = []

for index,match_id in tqdm(enumerate(match_id_list)):

    for team_innings,opponent_innings in zip(['first_innings','second_innings'],['second_innings','first_innings']):
        match_info = match_list_df[match_list_df['match_id']==match_id]
        team_info = match_info[match_info[team_innings]==match_info['team_statistics']]
        opponent_info = match_info[match_info[opponent_innings]==match_info['team_statistics']]

        team = team_info['team_statistics'].values[0].strip()
        opponent = opponent_info['team_statistics'].values[0].strip()
        location = team_info['location'].values[0].strip()
        ref_dt_np = team_info['date'].values[0]
        ref_date = cricutil.npdate_to_datetime(ref_dt_np)
        opponent_player_list = list()
        for boi in range(11):
            bowler = opponent_info['bowler_' + str(boi + 1)].values[0].strip()
            if bowler == 'not_bowled':
                break
            else:
                opponent_player_list.append(bowler)

        for bi in range(11):

            batsman = team_info['batsman_'+str(bi+1)].values[0].strip()
            if batsman == 'not_batted':
                break
            else:
                try:
                    feature_vector = fe.get_batsman_features_with_embedding(batsman,bi+1,opponent_player_list,team,opponent,location,ref_date=ref_date)
                    runs_scored = team_info['batsman_'+str(bi+1)+'_runs'].values[0]
                    if ref_date<test_start_dt:
                        feature_list_train.append(feature_vector)
                        target_list_train.append(runs_scored)
                    else:
                        feature_list_test.append(feature_vector)
                        target_list_test.append(runs_scored)
                except Exception as ex:
                    print(ex, ' for ',batsman, team, opponent, location, ' on ',ref_date.date() )

#print("pre-scaled values \n",np.stack(feature_list_train))
train_x = np.stack(feature_list_train)
train_y = np.stack(target_list_train)

test_x = np.stack(feature_list_test)
test_y = np.stack(target_list_test)

# pickle train_x, train_y,test_x,test_y,scaler
pickle.dump(train_x,open(os.path.join(TRAIN_TEST_DIR, batsman_runs_train_x),'wb'))
pickle.dump(train_y, open(os.path.join(TRAIN_TEST_DIR, batsman_runs_train_y), 'wb'))

pickle.dump(test_x, open(os.path.join(TRAIN_TEST_DIR, batsman_runs_test_x), 'wb'))
pickle.dump(test_y, open(os.path.join(TRAIN_TEST_DIR, batsman_runs_test_y), 'wb'))

outil.create_meta_info_entry('batsman_runs_train_xy', train_start,
                             str(cricutil.substract_day_as_datetime(test_start_dt, 1).date()),
                             file_list=[batsman_runs_train_x,
                                        batsman_runs_train_y])

outil.create_meta_info_entry('batsman_runs_test_xy', str(test_start_dt.date()),
                             str(test_end_dt.date()),
                             file_list=[batsman_runs_test_x,
                                        batsman_runs_test_y])

In [2]:
outil.use_model_from('dev')
batsman_group_emb_vec = fe.get_single_batsman_embedding('V Kohli',1,'India','Australia','Kolkata')

In [3]:
batsman_group_emb_vec

array([ 2.274468 ,  9.584267 , 10.5845585,  3.8093085, 10.524713 ,
       10.355957 ,  3.2759554,  0.       , 10.45765  , 10.458179 ],
      dtype=float32)

In [5]:
['a','b','c'].index('b')

1

In [17]:
!ls model

batsman_enc_map.pkl                  group_encode_model.json
batsman_group_encode_model.h5        group_encode_run_model.h5
batsman_group_encode_model.json      group_encode_run_model.json
batsman_group_encode_run_model.h5    inference_config.json
batsman_group_encode_run_model.json  loc_enc_map.pkl
country_enc_map.pkl                  location_enc_map_for_batsman.pkl
first_innings_base_columns.pkl       meta_info.json
first_innings_model.pkl              second_innings_base_columns.pkl
first_innings_model_base.pkl         second_innings_model.pkl
first_innings_selected_colindex.pkl  second_innings_model_base.pkl
first_innings_selected_features.pkl  second_innings_selected_colindex.pkl
group_encode_model.h5                second_innings_selected_features.pkl


In [19]:
pickle.load(open('model/first_innings_selected_features.pkl','rb'))

['team_score',
 'batsman_mean',
 'batsman_max',
 'batsman_sum',
 'bowler_mean',
 'bowler_sum']

In [21]:
pickle.load(open('model_ipl_dev/loc_enc_map.pkl','rb'))

{'unknown': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'Chennai': array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'Hyderabad': array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'Kochi': array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'Delhi': array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'Mumbai': array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'Kolkata': array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'Bangalore': array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'Jaipur': array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'Chandigarh': array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'Indore': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'Dharamsala': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'Visakhapatnam': array([